# Imports

In [1]:
from __future__ import print_function
import torch
import json

from data_loader import load_cifar10, get_class_names
from training_utils import train_model, continue_training

# CUDA

In [2]:
print(f"Is CUDA available? {torch.cuda.is_available()}")
!nvcc --version

Is CUDA available? True
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Initialization

In [4]:
# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_loader, test_loader, X_train, X_test, Y_train, Y_test = load_cifar10(batch_size=64, seed=42)
class_names = get_class_names()

Files already downloaded and verified
Files already downloaded and verified


# Train CNN

In [6]:
from model import BasicCNN
model = BasicCNN()
model = model.to(device)

print(f"Model device: {next(model.parameters()).device}")

Model device: cuda:0


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#metrics = train_model(
#    model=model,
#    train_loader=train_loader,
#    test_loader=test_loader,
#    model_name='cnn',
#    num_epochs=500,
#    device=device
#)

# Continue training

In [ ]:
import json
checkpoint_dir = '/content/drive/MyDrive/cnn_checkpoints'  # Replace with your actual path
with open(f'{checkpoint_dir}/cnn_training_metrics.json', 'r') as f:
       metrics = json.load(f)

print(f"Last completed epoch: {metrics['current_epoch']}")

# Continue training
metrics = continue_training(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    model_name='cnn',
    checkpoint_dir=checkpoint_dir,
    target_epochs=600,
    device=device
)

Last completed epoch: 200
